In [1]:
from google.colab import drive 

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 448 kB 47.8 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tqdm import tqdm

In [4]:
train = pd.read_csv('/content/gdrive/My Drive/bluehouse/train.csv', encoding = 'utf-8')
test = pd.read_csv('/content/gdrive/My Drive/bluehouse/test.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('/content/gdrive/My Drive/bluehouse/sample_submission.csv', encoding = 'utf-8')

In [5]:
train = train.dropna(how = 'any')
train['data'] = train['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test['data'] = test['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']

In [6]:
okt = Okt()

In [7]:
X_train = []
for sentence,i in zip(train['data'],tqdm(range(len(train['data'])))) :
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_train.append(temp_X)

100%|█████████▉| 39991/39992 [59:49<00:00, 11.14it/s]


In [8]:
X_test = []
for sentence in test['data']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_test.append(temp_X)

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [10]:
vocab_size = 30000
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [11]:
max_len = 500
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [12]:
y_train = to_categorical(train['category'])

In [13]:
model = Sequential()
model.add(Embedding(vocab_size, 120))
model.add(LSTM(120))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=15)

Epoch 1/15
313/313 [==============================] - 640s 2s/step - loss: 0.5132 - acc: 0.8009
Epoch 2/15
313/313 [==============================] - 638s 2s/step - loss: 0.2899 - acc: 0.8980
Epoch 3/15
313/313 [==============================] - 630s 2s/step - loss: 0.2226 - acc: 0.9221
Epoch 4/15
313/313 [==============================] - 622s 2s/step - loss: 0.1869 - acc: 0.9357
Epoch 5/15
313/313 [==============================] - 620s 2s/step - loss: 0.1684 - acc: 0.9404
Epoch 6/15
313/313 [==============================] - 617s 2s/step - loss: 0.2434 - acc: 0.9130
Epoch 7/15
313/313 [==============================] - 634s 2s/step - loss: 0.2111 - acc: 0.9232
Epoch 8/15
313/313 [==============================] - 625s 2s/step - loss: 0.1254 - acc: 0.9553
Epoch 9/15
313/313 [==============================] - 627s 2s/step - loss: 0.1049 - acc: 0.9621
Epoch 10/15
313/313 [==============================] - 619s 2s/step - loss: 0.1476 - acc: 0.9446
Epoch 11/15
313/313 [==================

In [16]:
y_pred = model.predict(X_test)
classes_x=np.argmax(y_pred,axis=1)
sample_submission['category'] = classes_x
sample_submission.to_csv('/content/gdrive/My Drive/bluehouse/submission.csv', encoding='utf-8', index = False)

In [ ]:
predict_x=model.predict(X_test) 
classes_x=np.argmax(predict_x,axis=1)